# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.sample(10)

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
59773,9000.0,6.971,mortgage,46200,0.350649,3,0,16200,low_risk
77420,18800.0,11.097,own,85000,0.647059,12,2,55000,high_risk
76693,20700.0,11.935,own,92900,0.677072,14,3,62900,high_risk
34245,11100.0,7.833,own,54300,0.447514,5,1,24300,low_risk
34570,9100.0,6.999,mortgage,46500,0.354839,3,0,16500,low_risk
13852,10100.0,7.397,own,50200,0.402390,4,1,20200,low_risk
37087,9600.0,7.211,mortgage,48500,0.381443,4,0,18500,low_risk
32780,9900.0,7.333,mortgage,49600,0.395161,4,0,19600,low_risk
73264,9200.0,7.026,rent,46700,0.357602,3,0,16700,low_risk
49386,10200.0,7.457,rent,50800,0.409449,4,1,20800,low_risk


# Split the Data into Training and Testing

In [30]:
# Create our features
from sklearn.preprocessing import LabelEncoder
X = df.drop(columns=['loan_status'])
le = LabelEncoder()
le.fit(X['homeowner'])
X['homeowner'] = le.transform(X['homeowner'])

# Create our target
y = df['loan_status'].replace({'high_risk':1, 'low_risk':0})

In [31]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [32]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

In [33]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=42)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [19]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [36]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [37]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [47]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.95276202927065

In [48]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[18699,    93],
       [   53,   539]])

In [49]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      0.91      1.00      0.95      0.91     18792
          1       0.85      0.91      1.00      0.88      0.95      0.90       592

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [50]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)

# View the count of target classes with Counter
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 56244, 1: 56244})

In [58]:
# Train the Logistic Regression model using the resampled data
model_naive_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_naive_ros.fit(X_resampled, y_resampled)

y_pred = model_naive_ros.predict(X_test)

In [60]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9954904617260939

In [61]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[18686,   106],
       [    2,   590]])

In [62]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      1.00      0.99     18792
          1       0.85      1.00      0.99      0.92      1.00      0.99       592

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [63]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 56244, 1: 56244})

In [64]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [65]:
# Calculated the balanced accuracy score
y_pred = model_smote.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9954904617260939

In [66]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18686,   106],
       [    2,   590]])

In [67]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      1.00      0.99     18792
          1       0.85      1.00      0.99      0.92      1.00      0.99       592

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [68]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 1908, 1: 1908})

In [70]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [71]:
# Calculate the balanced accuracy score
y_pred = model_cc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9846437673305489

In [72]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18691,   101],
       [   15,   577]])

In [73]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.97      1.00      0.98      0.97     18792
          1       0.85      0.97      0.99      0.91      0.98      0.97       592

avg / total       0.99      0.99      0.98      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [74]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 74592, 1: 73761})

In [76]:
# Train the Logistic Regression model using the resampled data
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smoteenn.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [78]:
# Calculate the balanced accuracy score
y_pred = model_smoteenn.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.995384033458746

In [79]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18682,   110],
       [    2,   590]])

In [80]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      1.00      0.99     18792
          1       0.84      1.00      0.99      0.91      1.00      0.99       592

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
